##Simple Gen AI app using langchain


In [4]:
import os 
from dotenv import load_dotenv
from langchain_ollama import OllamaEmbeddings
load_dotenv()
#langsmith tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN)TRACKING_V2']='True'
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')


In [57]:
#data ingestion--from the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader 
import bs4


In [6]:
#reading the data source
bs4_strainer = bs4.SoupStrainer(class_=("theme-doc-markdown markdown"))
loader = WebBaseLoader(web_paths=("https://docs.smith.langchain.com/observability",),bs_kwargs={"parse_only":bs4_strainer})
loader

In [7]:
#loading and convert it into document
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability'}, page_content='Observability Quick Start\nThis tutorial will get you up and running with our observability SDK by showing you how to\ntrace your application to LangSmith.\nIf you\'re already familiar with the observability SDK, or are interested in tracing more than just\nLLM calls you can skip to the next steps section,\nor check out the how-to guides.\nTrace LangChain or LangGraph ApplicationsIf you are using LangChain or LangGraph, which both integrate seamlessly with LangSmith,\nyou can get started by reading the guides for tracing with LangChain or tracing with LangGraph.\n1. Install Dependencies\u200b\nPythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai\n2. Create an API key\u200b\nTo create an API key head to the LangSmith settings page. Then click Create API Key.\n3. Set up your environment\u200b\nShellexport LANGSMITH_TRACING=trueexport LANGSMITH_API_KEY="<your-langsmith-api-ke

In [8]:
#dividing the documents into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000,chunk_overlap =200,add_start_index =True)
chunks = text_splitter.split_documents(docs)
chunks


[Document(metadata={'source': 'https://docs.smith.langchain.com/observability', 'start_index': 0}, page_content="Observability Quick Start\nThis tutorial will get you up and running with our observability SDK by showing you how to\ntrace your application to LangSmith.\nIf you're already familiar with the observability SDK, or are interested in tracing more than just\nLLM calls you can skip to the next steps section,\nor check out the how-to guides.\nTrace LangChain or LangGraph ApplicationsIf you are using LangChain or LangGraph, which both integrate seamlessly with LangSmith,\nyou can get started by reading the guides for tracing with LangChain or tracing with LangGraph.\n1. Install Dependencies\u200b\nPythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai\n2. Create an API key\u200b\nTo create an API key head to the LangSmith settings page. Then click Create API Key.\n3. Set up your environment\u200b"),
 Document(metadata={'source': 'https://docs.smith.langchain.com

In [9]:
#converting chunks into vectors
embeddings = (OllamaEmbeddings(model = "all-minilm"))




In [10]:
#storing the vectors in vectordb (FAISS)
import faiss
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents=chunks,embedding=embeddings)

In [56]:
#query from a vectordb
query = "trace your application in langsmith"
result = vectorstore.similarity_search(query)
result[0].page_content

"Observability Quick Start\nThis tutorial will get you up and running with our observability SDK by showing you how to\ntrace your application to LangSmith.\nIf you're already familiar with the observability SDK, or are interested in tracing more than just\nLLM calls you can skip to the next steps section,\nor check out the how-to guides.\nTrace LangChain or LangGraph ApplicationsIf you are using LangChain or LangGraph, which both integrate seamlessly with LangSmith,\nyou can get started by reading the guides for tracing with LangChain or tracing with LangGraph.\n1. Install Dependencies\u200b\nPythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai\n2. Create an API key\u200b\nTo create an API key head to the LangSmith settings page. Then click Create API Key.\n3. Set up your environment\u200b"

In [ ]:
#Initializing a language model
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model ="llama3")


In [ ]:
#Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
    <question>{input}</question>
    <context>{context}</context>"""
)
document_chain = create_stuff_documents_chain(llm,prompt)

In [50]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context:\n    <question>{input}</question>\n    <context>{context}</context>'), additional_kwargs={})])
| OllamaLLM(model='llama3')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [ ]:
question = "How to trace your application in langsmith"
text =document_chain.invoke({"input": question, "context": chunks})
print(text)

To trace your application in langsmith, you need to:

1. Wrap OpenAI calls using `wrap_openai` (Python) or `wrapOpenAI` (TypeScript)
2. Use the `[traceable]` decorator (Python or TypeScript) to trace your entire application instead of just LLM calls.

For example, if you want to trace an OpenAI call:

```python
from openai import OpenAI
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(OpenAI())
...
```

Or if you want to trace the entire pipeline:

```python
@traceable
def rag(question):
    ...
```

Note: The code above should be used in your application's `rag` function.


In [52]:
#Retrieval chain
retriever = vectorstore.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [53]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001707E8A6140>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context:\n    <question>{input}</question>\n    <context>{context}</contex

In [54]:
## Get the response from the LLM
response =retrieval_chain.invoke({"input":question})

In [55]:
print(response['answer'])

To trace your application in Langsmith, you can follow these steps:

1. Install the necessary dependencies using pip or yarn.
2. Create an API key on the LangSmith settings page and create a tracing project.
3. Set up your environment by exporting the `LANGSMITH_TRACING` and `LANGSMITH_API_KEY` variables.

When you call your application, such as `rag("where did harrison work")`, this will produce a trace of just the OpenAI call in LangSmith's default tracing project.

Note that you'll also need to define your application by instrumenting it with an LLM call.
